<h1 id="tocheading">Table of Contents</h1>
<div id="toc"></div>

In [3]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

# Chapter 4: Working with Key/Value Pairs

## Pair RDDs
* Spark provides special operations on RDDs containing key/value pairs - these RDDs are called pair RDDs
* We can use the <code>map()</code> function to create a pair RDD as shown below
* Pair RDDs are still RDDs (of Tuple2 objects in Java/Scala or of Python tuples) 

### Example: Creating a pair RDD using the first word as the key

In [27]:
val lines = sc.textFile("name_example.txt")
val rddpair = lines.map(x => (x.split(" ")(0),x))


println("Results from map: ")
rddpair.collect().foreach(println)

println("\nResults from lookup: ")
rddpair.lookup("her").foreach(println)

Results from map: 
(his,his name is pat)
(his,his name is peter)
(his,his name is olaf)
(her,her name is Joanne)
(her,her name is Therese)
(they,they work at algebraix in Encintas )
(they,they like to eat yogurt )

Results from lookup: 
her name is Joanne
her name is Therese


## Transformations on Pair RDDs

### Transformation on one pair RDD
* Pair RDDs are allowed to use all the transformations available to standard RDDs
* The same rules apply from "Passing Functions to Spark" from Chapter 3
    - Since pair RDDs contain tuples, we need to pass functions that operate on tuples rather than on individual elements
* <code>reduceByKey(func)</code>: Combine values with the same key 
    - Note that calling <code>reduceByKey()</code> and <code>foldByKey()</code> will automatically perform combining locally on each machine before computing global totals for each key and hte user does not need to specify a combiner
    - The more general <code>combineByKey</code> interface allows you to customize combining behavior
    
* <code>groupByKey()</code>: Group values with the same key
* <code>combineByKey(createCombiner,mergeValue,mergeCombiners,partitioner)</code>: Combine values with the same key using a different result type
    - <code>combineByKey()</code> is the most general of the per-key aggregation functions
    - Most of the other per-key combiners are implemented using it
    - Like <code>aggregate</code>, <code>combineByKey()</code> allows the user to return values that are not hte same type as our input data 
    - Understsanding <code>combineByKey()</code> in depth:<br>
    For each element in the partition, the element either has a key it hasn't seen before or has the same key as a previous element:
        - If it's a new element, <code>combineByKey()</code> uses a function we provide, called <code>createCombiner()</code> to create the initial value for the accumulator on the key (**Note: This happens the first time a key is found in each partition rather than only the first time the key is found in the RDD**)
        - If it is a value we have seen before while processing that partition, it will instead use the provided function, <code>mergeValue()</code>, with the current value for the accumulator for that key and the new value
        - <code>mergeCombiners()</code>: merges the accumulators of the same key across all partitions
* <code>mapValues(func)</code>: Apply a function to each value of a pair RDD without changing they key
* <code>flatMapValues(func)</code>: Apply a function taht returns an iterator to each value of a pair RDD, and for each element returned produce a key/value entry with the old key. Often used for tokenization 
* <code>keys()</code>: Return an RDD of just the keys
* <code>values()</code>: Return an RDD of just the values
* <code>sortByKey()</code>: Return an RDD sorted by the key

### Transformation on two pair RDDs
* <code> rdd.subtractByKey(other)</code>: Remove elements with a key present in the other RDD
* <code> rdd.join(other)</code>: Perform an inner join between two RDDs
* <code> rdd.rightOuterJoin(other)</code>: Perform a join between two RDDs where the key must be present in the first RDD 
* <code> rdd.leftOuterJoin(other)</code>: Perform a join between two RDDs where the key must be present in the other RDD
* <code> rdd.cogroup(other)</code>: Group data from both RDDs sharing the same key. 
    - <code>cogroup()</code> over two RDDs sharing the same key type, K, with the respective value types V and W returns <code>RDD[(K,(Iterable[V]),Iterable[W]))]</code>
    - If one of the RDDs doesn't have elements for a given key that is present in the other RDD, the corresponding <code>Iterable</code> is simply empty
    - can be used to implement intersection by key
    - can work on three or more RDDs at once


### Example on one pair RDD

In [121]:
val rdd = sc.parallelize(List((3,2),(1,4),(3,6),(3,1)))
println("rdd: " + rdd.collect().mkString(","))

//add values with the same key 
println("reduceByKey example: " + rdd.reduceByKey(_ + _).collect().mkString(","))

//groupByKey() returns an iterable in the values, the mapValues function below converts the iterable to a list
println("groupByKey example: " + rdd.groupByKey().mapValues(x => x.toList).collect().mkString(","))

//mapValues example
println("mapValues example: " + rdd.mapValues(_ + 1).collect().mkString(","))

//flatMap example:
// rdd.flatMapValues(x => (x to 3)).collect()
println("flatMapValues example:")
println("\tbefore flatMapValues:" + rdd.mapValues(x=>(x to 3)).collect().mkString(","))
println("\tafter flatMapValues: " + rdd.flatMapValues(x=>(x to 3)).collect().mkString(","))
println("keys example: " + rdd.keys.collect().mkString(","))
println("values example: " + rdd.values.collect().mkString(","))
println("sortByKey example: " + rdd.sortByKey().collect().mkString(","))


rdd: (3,2),(1,4),(3,6),(3,1)
reduceByKey example: (1,4),(3,9)
groupByKey example: (1,List(4)),(3,List(2, 6, 1))
mapValues example: (3,3),(1,5),(3,7),(3,2)
flatMapValues example:
	before flatMapValues:(3,Range(2, 3)),(1,Range()),(3,Range()),(3,Range(1, 2, 3))
	after flatMapValues: (3,2),(3,3),(3,1),(3,2),(3,3)
keys example: 3,1,3,3
values example: 2,4,6,1
sortByKey example: (1,4),(3,2),(3,6),(3,1)


In [32]:
type((1,2))

Name: Compile Error
Message: <console>:1: error: identifier expected but '(' found.
       type((1,2))
           ^
StackTrace: 